# Ejemplo de código de optimizador de prompts
Este código muestra cómo optimizar un prompt para un modelo de lenguaje GPT-3.5, utilizando GPT-4 como optimizador. El codigo esta basado en el paper [Large Language Models as Optimizers. Yang et al. (2023).](https://doi.org/10.48550/arXiv.2309.03409)


## 1.- Setup inicial

### 1.1- Instalar librerías

In [ ]:
#! pip install openai
#! pip install tenacity
#! pip install python-dotenv

### 1.2.- Cargar librerías

In [ ]:
import openai
import os
import numpy as np
from datetime import datetime
from dotenv import load_dotenv
from tenacity import retry, wait_random_exponential, stop_after_attempt

### 1.3.- Variables de entorno

In [ ]:
# Load secrets and config from .env file
load_dotenv()

# OpenAI API
openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = os.getenv("OPENAI_API_VERSION")
openai.api_key = os.getenv("OPENAI_API_KEY")
embedding_model = os.getenv("OPENAI_EMBEDDING_MODEL")
print("OpenAI API key: {}".format(openai.api_key[:5] + '...' + openai.api_key[-5:]))
print("OpenAI API base: {}".format(openai.api_base))
print("OpenAI API version: {}".format(openai.api_version))
print("OpenAI API type: {}".format(openai.api_type))

# Model endpoint names
gpt35_model = os.getenv("OPENAI_GPT35_MODEL")
gpt35_16k_model = os.getenv("OPENAI_GPT35_16K_MODEL")
gpt4_model = os.getenv("OPENAI_GPT4_MODEL")
print("GPT-3.5-Turbo model: {}".format(gpt35_model))
print("GPT-3.5-Turbo-16k model: {}".format(gpt35_16k_model))
print("GPT-4 model: {}".format(gpt4_model))

### 1.4.- Clase para logs

In [ ]:
class Logger:
    def __init__(self, log_file=None, log_dir='./logs'):
        if not os.path.exists(log_dir):
            os.makedirs(log_dir)
        
        if log_file is None:
            timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
            log_file = f"log_{timestamp}.log"
        
        self.log_file = os.path.join(log_dir, log_file)

    def _write_log(self, level, msg, verbose):
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        log_entry = f"{timestamp} [{level}] {msg}\n"
        with open(self.log_file, "a") as f:
            f.write(log_entry)
        if verbose:
            print(log_entry.strip())

    def info(self, msg, verbose=False):
        self._write_log("INFO", msg, verbose)

    def warn(self, msg, verbose=False):
        self._write_log("WARN", msg, verbose)

    def error(self, msg, verbose=True):
        self._write_log("ERROR", msg, verbose)
        
logger = Logger()
logger.info("Logger initialized", verbose=True)

### 1.5.- Clase para generación de texto con modelos GPT

In [ ]:
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(3))
def generate_text(prompt, model=gpt35_model, messages=[], max_tokens=600, temperature=0.5, top_p=1.0, frequency_penalty=0.0, presence_penalty=0.0, stop=None):
    _messages = []
    _messages.extend(messages)
    _messages.append({"role": "user", "content": prompt})
    
    _log_message = "\n\n============================ PROMPT ============================\n"
    for message in _messages:
        _log_message += f"{message['role']}: {message['content']}\n"
    logger.info(_log_message)
        
    response = openai.ChatCompletion.create(
        engine=model,
        messages=_messages,
        max_tokens=max_tokens,
        temperature=temperature,
        top_p=top_p,
        frequency_penalty=frequency_penalty,
        presence_penalty=presence_penalty,
        stop=stop
    )
    _log_message = "\n\n============================ RESPONSE ============================\n"
    _log_message += f"{response}\n"
    logger.info(_log_message)
    return response["choices"][0]["message"]["content"]

## 2.- Componentes de una prompt

### 2.1.- Prompt simple

In [ ]:
prompt = "Que es una prompt?"
print(generate_text(prompt))

In [ ]:
prompt = "En una oración. Que es una prompt?"
print(generate_text(prompt))

In [ ]:
prompt = "Prompt: Un prompt es una señ"
print(generate_text(prompt))

In [ ]:
prompt = """User: Prompt
AI: Una prompt es una señal o indicación que se da a un usuario para realizar una acción, comúnmente usada en programación y sistemas operativos.
User: Java
AI:"""
print(generate_text(prompt))

### 2.2.- Mensajes previos (contexto)

In [ ]:
messages = [
    {"role": "user", "content": "Prompt"},
    {"role": "assistant", "content": "Un prompt es una señal o indicación en un programa de computadora que solicita al usuario que introduzca un comando o información."}
    ]
prompt = "Java"
print(generate_text(prompt, messages=messages))

In [ ]:
messages = [
    {"role": "user", "content": "Prompt"},
    {"role": "assistant", "content": "Un prompt es una señal o indicación en un programa de computadora que solicita al usuario que introduzca un comando o información."},
    {"role": "user", "content": "Java"},
    {"role": "assistant", "content": "Java es un lenguaje de programación de alto nivel, orientado a objetos y diseñado para desarrollar software para múltiples plataformas."}
    ]
prompt = "JavaScript"
print(generate_text(prompt, messages=messages))

### 2.3.- System message

In [ ]:
system_message = "Responde en una sola oración con una descripcion del concepto ingresado por el usuario. Responde siempre en español."
messages = [{"role": "system", "content": system_message}]
prompt = "Prompt"
print(generate_text(prompt, messages=messages))

In [ ]:
prompt = "Java"
print(generate_text(prompt, messages=messages))

## 3.- Parametros de GPT

### 3.1.- Stop

In [ ]:
prompt = "Que es una prompt?"
print(generate_text(prompt, temperature=0))

In [ ]:
prompt = "Que es una prompt?"
print(generate_text(prompt, temperature=0, stop=["."]))

### 3.2.- Temperature

In [ ]:
TEMPERATURE_STEPS = 5
CONSISTENCY_ITERATIONS = 3

answers = []
for i in range(TEMPERATURE_STEPS):
    temperature = np.interp(i, [0, TEMPERATURE_STEPS-1], [0, 1.5])
    for j in range(CONSISTENCY_ITERATIONS):
        answer = generate_text("En una oracion. Que es una prompt?", temperature=temperature)
        print(f"{j} - Temperature: {temperature:.2f}, Answer: {answer}")
        answers.append({"temperature": temperature, "consistency": j, "answer": answer})

### 3.2.- Top P

In [ ]:
TOP_P_STEPS = 5
CONSISTENCY_ITERATIONS = 3

answers = []
for i in range(TOP_P_STEPS):
    top_p = np.interp(i, [0, TOP_P_STEPS-1], [0, 1.0])
    for j in range(CONSISTENCY_ITERATIONS):
        answer = generate_text("En una oracion. Que es una prompt?", top_p=top_p)
        print(f"{j} - Top_p: {top_p:.2f}, Answer: {answer}")
        answers.append({"top_p": top_p, "consistency": j, "answer": answer})

### 3.2.- Frequency penalty

In [ ]:
FREQ_PENALTY_STEPS = 5
CONSISTENCY_ITERATIONS = 3

answers = []
for i in range(FREQ_PENALTY_STEPS):
    frequency_penalty = np.interp(i, [0, FREQ_PENALTY_STEPS-1], [0, 1.0])
    for j in range(CONSISTENCY_ITERATIONS):
        answer = generate_text("En una oracion. Que es una prompt?", frequency_penalty=frequency_penalty)
        print(f"{j} - Frequency penalty: {frequency_penalty:.2f}, Answer: {answer}")
        answers.append({"frequency_penalty": frequency_penalty, "consistency": j, "answer": answer})

### 3.3.- Presence penalty

In [ ]:
PRESENCE_PENALTY_STEPS = 5
CONSISTENCY_ITERATIONS = 3

answers = []
for i in range(PRESENCE_PENALTY_STEPS):
    presence_penalty = np.interp(i, [0, PRESENCE_PENALTY_STEPS-1], [0, 1.0])
    for j in range(CONSISTENCY_ITERATIONS):
        answer = generate_text("En una oracion. Que es una prompt?", presence_penalty=presence_penalty)
        print(f"{j} - Presence penalty: {presence_penalty:.2f}, Answer: {answer}")
        answers.append({"presence_penalty": presence_penalty, "consistency": j, "answer": answer})